# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/envs/finetuning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.60it/s]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、强、刚、动、健等含义。乾卦的卦辞是“元、亨、利、贞”，表示事物发展顺利、充满活力和希望。

乾卦的阳爻代表刚强、积极、进取的精神，阴爻则代表柔顺、配合、 yield 的态度。乾卦象征着阳刚之气的充实、生长和发展，是《易经》中最积极、最具有建设性的卦象之一。

在股市中，乾卦的阳爻象征着市场的上涨和繁荣，阴爻则象征着市场的下跌和疲软。因此，乾卦的总体趋势是向上发展，预示着股市中的投资者将会在市场上取得收益。

在个人生活中，乾卦的阳爻象征着积极、进取、勇敢和坚定等特质，阴爻则象征着耐心、合作、谦虚和柔软等态度。因此，乾卦鼓励人们在生活中要积极、进取、勇敢和坚定，同时也要有耐心、合作、谦虚和柔软的态度，以实现个人生活的成功和发展。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第二卦，也是八卦之一。讼卦由两个阳爻夹一个阴爻构成，象征着争斗、诉讼、矛盾等含义。讼卦的卦辞是“损、贞”，表示争斗虽然可以解决，但结果却是损耗和消耗。

讼卦的阳爻代表阳刚之气的充实、坚定和勇敢，阴爻则代表柔顺、配合、退让的态度。讼卦象征着阳刚之气的充实和坚定，但也提醒人们要注意柔顺、配合、退让等态度，以避免争斗和诉讼的发生。

在股市中，讼卦的阳爻象征着市场的上涨和盈利，阴爻则象征着市场的下跌和亏损。因此，讼卦的总体趋势是向下跌落，预示着股市中的投资者可能会面临损失。

在个人生活中，讼卦的阳爻象征着坚定、勇敢和勇敢等特质，阴爻则象征着退让、配合、谦虚和柔软等态度。因此，讼卦鼓励人们在面对矛盾和争斗时要坚定、勇敢，同时也要有退让、配合、谦虚和柔软的态度，以实现个人生活的成功和发展。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240321_110036"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦为乾为天，为乾，六阳之卦。乾卦代表天、干燥、闭塞、蒙昧等现象。在人事上象征着尊贵、显赫、刚健、震动等。乾卦象征着天行健，君子以自强不息。在占卜中，乾卦预示着天时干燥，君子应加强德行，在天时上应敬德，敬天保民，安身立命，敬德修身，事业可成。在经商方面，乾卦预示着商业行情将出现波动，商贾应谨慎行事，注意资金安全。在决策方面，乾卦提醒要遵循天时，加强自身修养，预示事物发展变化艰难，需要坚持和努力。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240321_110036）：
[gMASK]sop 解释下乾卦是什么？ 乾卦为乾为天，为干燥，为天，为刚强，为正义。为天之纯阳，万物之始，具有刚强之德。代表刚健不恒，物极必反。在人事方面，意味着刚强、正直，但需注意物极必反。在经商方面，需注意物极必反，注意适可而止。在军事方面，代表正义、勇敢、果敢，但需谨慎，避免盲目冒险。在恋爱婚姻方面，意味着刚强、正直，但需谨慎，避免过于刚强。在决策方面，提醒要刚强正直，但需谨慎，适可而止，避免物极必反。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦为周易六十四卦之一，卦象为雷交震，代表诉讼、争端、纷争等事。卦象分析表明，君子观此卦象，应明察争端所在，以和平手段化解纷争，安分守己。运势方面，所遇困境需谨慎应对，勿争强斗胜，应保持平和心态。事业方面需注意沟通，避免诉讼。经商方面要遵守商业道德，诚信经营，避免纠纷。求名方面要谦逊有礼，努力争取机会。婚恋中要注意处理矛盾，避免诉讼，家和为美。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240321_110036）：
[gMASK]sop 周易中的讼卦是什么卦象, 讼卦为上坎下离, 离为火,坎为水,火克水,先寒后温,先明后暗。卦象中水在火上,表示矛盾难以解决,需以智慧化解。运势上矛盾重重的局面,需谨慎处理,以和平手段化解矛盾。事业上有矛盾和冲突,需谨慎处理,避免影响事业的发展。经商需谨慎处理,避免商业纠纷。求名需耐心等待,避免急功近利。婚恋中矛盾重重的局面,需耐心化解,避免影响感情。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦为《易经》卦象之一，为水泽上的卦象，代表师兵之象。象征军队、兵旅、旅店等。卦象为：上为水，下为泽，中间为兵马。君子观此卦象，取法于师，以明德示人。师卦揭示兵者必归，众归一，敌归我也。师卦象征师出无名，需明德归服众人。运势起伏不定，需明德归服，家室、身体、事业需以德服人。经商需有计划，勿贪心，需量力而行。求名需诚实、谦虚、勤学。婚恋需相互了解，决策需量力而行，谨防师门不测。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240321_110036）：
[gMASK]sop 师卦是什么？ 师卦为乾上坎下，为师卦，代表师兵之象。表示众与寡共战，师出无名，师临不测。师卦象征兵起众共战，兵临未战，吉利的卦象。君子观此卦象，以时行事，不可妄动。师卦代表师兵之象，代表众与寡共战，师出无名，师临不测。师卦象征兵起众共战，兵临未战，吉利的卦象。君子观此卦象，以时行事，不可妄动。运势大起大落，师出无名，师临不测，需谨慎行事。事业须有计划、耐心、合作，避免盲目冒险。经商需有耐心，合作共事，避免独断独行。婚恋中要真诚、耐心，避免冲动。决策时应谨慎行事，不可轻举妄动，但也不可过于谨慎。
